In [9]:
import os
import numpy as np
import pandas as pd
from ecog_ucsf.block import read_block
from scipy.signal import decimate

In [2]:
# Functions that return channel means and standard deviations,
# calculated over the whole file, a portion of the file defined by
# the 'baseline' textgrid label, or the portion of the file that
# is detected to be the quietest 5-second interval.
#
# Return the values for channels 0-255; 'channel' 256 is the mean and
# standard deviation of all channels.

def whole_file(b):
    '''Calculate mean and std over whole file.'''
    return pd.DataFrame({
        'mean': np.append(np.nanmean(b.data, axis=1), np.nanmean(b.data)),
        'std': np.append(np.nanstd(b.data, axis=1), np.nanstd(b.data)),
        'channel': np.arange(257),
        'method': 'whole'
    })

def baseline(b):
    '''Calculate mean and std from predefined baseline region.'''
    btier = b.labels['baseline']
    lab = btier[btier.baseline == 'baseline']
    idx = np.round(
        np.array([lab.t1.iloc[0], lab.t2.iloc[0]]) * b.datarate
    ).astype(np.int32)
    d = b.data[:, np.arange(*idx)]
    return pd.DataFrame({
        'mean': np.append(np.nanmean(d, axis=1), np.nanmean(d)),
        'std': np.append(np.nanstd(d, axis=1), np.nanstd(d)),
        'channel': np.arange(257),
        'method': 'baseline'
    })

def quietest(b):
    '''Calculate mean and std from quietest 5-second region (as defined by variance).'''
    winsize = np.round(b.datarate * 5).astype(np.int32)
    activity = np.convolve(
        np.nanvar(b.data, axis=0),
        np.ones(winsize),
        'valid'
    )
    minidx = activity.argmin()
    d = b.data[:, np.arange(minidx, minidx + winsize)] 
    return pd.DataFrame({
        'mean': np.append(np.nanmean(d, axis=1), np.nanmean(d)),
        'std': np.append(np.nanstd(d, axis=1), np.nanstd(d)),
        'channel': np.arange(257),
        'method': 'quietest'
    })

In [3]:
basedir = '../../ecog/datasets'
blockdirs = ['GP31_B40', 'GP31_B34']   # <-- define your own set here
subdir = 'HilbAA_70to150_8band'

In [10]:
# Loop over block dirs and read data. Combine frequency bands and downsample, then
# calculate mean and standard deviation.
blocks = {}
results = []
for f in blockdirs:
    b = read_block(
        os.path.join(basedir, f),
        subdir,
        converter=lambda chan: decimate(
            chan.mean(axis=1), q=10, axis=0, zero_phase=True
        ),
        labeltiers=['phone', 'word', 'baseline']
    )
    blocks[f] = b
    results.append(whole_file(b))
    results.append(baseline(b))
    results.append(quietest(b))
rdf = pd.concat(results)

/Users/ronald/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: Mean of empty slice
/Users/ronald/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1427: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Users/ronald/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:26: RuntimeWarning: Mean of empty slice
/Users/ronald/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:43: RuntimeWarning: Mean of empty slice


In [11]:
rdf

,channel,mean,method,std
0,0,1.041462,whole,0.262820
1,1,0.880343,whole,0.217455
2,2,0.543250,whole,0.135252
3,3,0.711413,whole,0.328143
4,4,1.141778,whole,0.370517
5,5,1.082052,whole,0.326661
6,6,0.507126,whole,0.193888
7,7,0.347908,whole,0.150243
8,8,0.329823,whole,0.148972
9,9,0.338875,whole,0.196243
